In [1]:
%load_ext sql
%sql sqlite://

Create tables for nodes and edges. The data model is a graph, where edges can have labels and nodes can have values.

We simulate labels on nodes by a labeled self-loop. We use the node value to specify a type, rather than a concrete value.

In [2]:
%%sql
CREATE TABLE nodes(
    id INT PRIMARY KEY,
    value VARCHAR
);
CREATE TABLE edges(
    id INT PRIMARY KEY,
    source INT NOT NULL,
    label VARCHAR,
    target INT NOT NULL
)

 * sqlite://
Done.
Done.


[]

Fill the tables with some values.

In [3]:
%%sql
INSERT INTO nodes VALUES
    (0, NULL),
    (1, "INT"),
    (2, NULL),
    (3, "INT"),
    (4, NULL),
    (5, "DATE");

INSERT INTO edges VALUES
    (0, 0, "a", 1),
    (1, 2, "a", 3),
    (2, 4, "b", 5),
    (10, 0, "A", 0),
    (11, 2, "A", 2),
    (12, 4, "B", 4)

 * sqlite://
6 rows affected.
6 rows affected.


[]

List the value of each node.

In [4]:
%%sql
SELECT id, value FROM nodes

 * sqlite://
Done.


id,value
0,None
1,INT
2,None
3,INT
4,None
5,DATE


Get a list of (value, label, value) triples for each edge.

In [5]:
%%sql
SELECT s.value AS source, e.label, t.value AS target
FROM nodes s JOIN edges e JOIN nodes t
ON s.id = e.source AND e.target = t.id

 * sqlite://
Done.


source,label,target
None,a,INT
None,a,INT
None,b,DATE
None,A,None
None,A,None
None,B,None


**Schema validation**

We assume the following schema with two node types in the property graph model:

```
NODE :A { a INT };
NODE :B { b DATE };
```

This schema can be expressed by a set of FOL-rules. Here, we write a SQL query for each FOL rule, such that the FOL rule is true iff the last column of the query result contains only 1s.

*The following rules capture the vocabulary for nodes and edges.*

1a) Every node has a self-loop labeled `A` or `B` or has value `INT` or `DATE`.

In [6]:
%%sql
SELECT n.id, e.label, n.value, e.label = "A" OR e.label = "B" OR n.value = "INT" OR n.value = "DATE"
FROM nodes n LEFT OUTER JOIN edges e
ON e.source = n.id AND e.target = n.id

 * sqlite://
Done.


id,label,value,"e.label = ""A"" OR e.label = ""B"" OR n.value = ""INT"" OR n.value = ""DATE"""
0,A,None,1
1,None,INT,1
2,A,None,1
3,None,INT,1
4,B,None,1
5,None,DATE,1


1b) Every self-loop has label `A` or `B`.

In [7]:
%%sql
SELECT id, label, label = "A" OR label = "B"
FROM edges
WHERE source = target

 * sqlite://
Done.


id,label,"label = ""A"" OR label = ""B"""
10,A,1
11,A,1
12,B,1


1c) For every non-self-loop edge `e`:
- The source node `s` has a self-loop with the specified label `slabel`
- The edge `e` has the specified label `elabel`
- The target node `t` has the specified value `tvalue`

In [8]:
%%sql
SELECT
    e.id,
    se.label AS slabel,
    e.label AS elabel,
    t.value AS tvalue,
    (se.label = "A" AND e.label = "a" AND t.value = "INT"
    OR se.label = "B" AND e.label = "b" AND t.value = "DATE")
FROM
    (nodes s LEFT OUTER JOIN edges se
    ON se.source = s.id AND se.target = s.id)
    JOIN edges e
    ON e.source = s.id AND e.source != e.target
    JOIN nodes t
    ON e.target = t.id

 * sqlite://
Done.


id,slabel,elabel,tvalue,"(se.label = ""A"" AND e.label = ""a"" AND t.value = ""INT"" OR se.label = ""B"" AND e.label = ""b"" AND t.value = ""DATE"")"
0,A,a,INT,1
1,A,a,INT,1
2,B,b,DATE,1


*The following rules capture that properties are mandatory.*

2a) Every `A`-node has an outgoing `a`-edge to a node with value `INT`.

In [9]:
%%sql
SELECT
    s.id AS slabel,
    e.label AS elabel,
    t.value AS tvalue,
    e.label = "a" AND t.value = "INT"
FROM
    (nodes s JOIN edges se
    ON se.source = s.id AND se.target = s.id AND se.label = "A")
    LEFT OUTER JOIN edges e ON e.source = s.id AND e.label = "a"
    LEFT OUTER JOIN nodes t ON e.target = t.id

 * sqlite://
Done.


slabel,elabel,tvalue,"e.label = ""a"" AND t.value = ""INT"""
0,a,INT,1
2,a,INT,1


2b) Every `B`-node has an outgoing `b`-edge to a node with value `DATE`.

In [10]:
%%sql
SELECT
    s.id AS slabel,
    e.label AS elabel,
    t.value AS tvalue,
    e.label = "b" AND t.value = "DATE"
FROM
    (nodes s JOIN edges se
    ON se.source = s.id AND se.target = s.id AND se.label = "B")
    LEFT OUTER JOIN edges e ON e.source = s.id AND e.label = "b"
    LEFT OUTER JOIN nodes t ON e.target = t.id

 * sqlite://
Done.


slabel,elabel,tvalue,"e.label = ""b"" AND t.value = ""DATE"""
4,b,DATE,1


*The following rules capture that properties are single-valued.*

*The following rules capture mutual exclusion between different node types.*